In [1]:
import enum
import importlib
import inspect
import os
import shutil

from typing import List
from IPython.display import HTML
from sflkit.analysis.spectra import Line

from sflkit.analysis import analysis_type, factory
from sflkit import *
from sflkit.color import ColorCode
from sflkit import instrument_config, analyze_config
from sflkit.config import Config

from avicenna import *

## A faulty Program

First, we need a faulty program. We chose an implementation of the `middle(x, y, z)` function that returns the *middle* number of its three arguments. For example, `middle(1, 3, 2)` should return 2 because `1 < 2` and `2 < 3`. We introduced a fault in this implementation of `middle` that occurs in line 7 `m = y`. 

In [2]:
#!/bin/python3

"""
This file contains the code under test for the example bug.
The sqrt() method fails on x <= 0.
"""
from math import cos as rcos
from math import sin as rsin
from math import tan as rtan


def sqrt(x):
    """Computes the square root of x, using the Newton-Raphson method"""
    if x == 0:
        return 0
    assert x > 0
    approx = None
    guess = x / 2
    while approx != guess:
        approx = guess
        guess = (approx + x / approx) / 2
    return approx


def tan(x):
    return rtan(x)


def cos(x):
    return rcos(x)


def sin(x):
    return rsin(x)


def main(arg):
    return eval(arg)

In [3]:
main('sqrt(9)')

3.0

In [4]:
import string
from typing import List


grammar_calc = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><one_nine><maybe_digits><maybe_frac>", 
                 #"-0",
                 "0"],
    "<maybe_minus>": ["",
                      "-"
                      ],
    "<maybe_frac>": ["", ".<digits>"],
    "<one_nine>": [str(num) for num in range(1, 10)],
    "<digit>": [digit for digit in string.digits],
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
}

In [5]:
# from avicenna.avix import *

# AviX.create_event_file(inp='sqrt(9)',
#                        instrumented_function='main',
#                        conversion_func=None,
#                        event_path='rsc/event_file'
#                        )



In [6]:
put = 'calculator.py'
tmp_py = 'rsc/instrumented.py'
language='python'
predicates='line'
metrics='Tarantula'
#passing='event-files/0,event-files/1'
failing='event-files/event_file'
def get_config():
    return Config.create(path=put,
                         working=tmp_py,
                         language=language,
                         predicates=predicates,
                         metrics=metrics,
                         failing=failing
                         )
    
def instrument(out=True):
    instrument_config(get_config())
    if out:
        with open(tmp_py, 'r') as fp:
            print(fp.read())

In [7]:
instrument()

sflkit :: INFO     :: I found 16 events in calculator.py.
sflkit :: INFO     :: I found 16 events in calculator.py.


import sflkitlib.lib
sflkitlib.lib.add_line_event(0)
from math import cos as rcos
sflkitlib.lib.add_line_event(1)
from math import sin as rsin
sflkitlib.lib.add_line_event(2)
from math import tan as rtan

def sqrt(x):
    sflkitlib.lib.add_line_event(3)
    if x == 0:
        sflkitlib.lib.add_line_event(4)
        return 0
    sflkitlib.lib.add_line_event(5)
    assert x > 0
    sflkitlib.lib.add_line_event(6)
    approx = None
    sflkitlib.lib.add_line_event(7)
    guess = x / 2
    sflkitlib.lib.add_line_event(8)
    while approx != guess:
        sflkitlib.lib.add_line_event(9)
        approx = guess
        sflkitlib.lib.add_line_event(10)
        guess = (approx + x / approx) / 2
    sflkitlib.lib.add_line_event(11)
    return approx

def tan(x):
    sflkitlib.lib.add_line_event(12)
    return rtan(x)

def cos(x):
    sflkitlib.lib.add_line_event(13)
    return rcos(x)

def sin(x):
    sflkitlib.lib.add_line_event(14)
    return rsin(x)

def main(arg):
    sflkitlib.lib.add_line

In [8]:
def analyzer_conf(conf: Config, factory):
    analyzer = Analyzer(irrelevant_event_files=conf.failing, 
                        relevant_event_files=conf.passing,
                        factory=factory)
    return analyzer

In [9]:
factory = factory.LineFactory()
line_event = analysis_type.AnalysisType(0)
line_event

LINE

In [10]:
analyzer = analyzer_conf(get_config(), factory=factory)

In [11]:
# results = analyzer.analyze()
# results

# analyzer needs event files 
analyzer.analyze()

In [12]:
coverage: List[Line] = analyzer.get_coverage()
coverage = {line.line for line in coverage}
print(coverage)

{38, 14, 16, 17, 18, 19, 20, 21, 22}


In [13]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [14]:
# from avicenna.avix import AviX
# AviX.create_event_file( instrumented_function='main',
#                         #instr_path=instrumented_file_path,
#                         inp='sqrt(9)', 
#                         conversion_func=None, 
#                         event_path='rsc/event_file',
# )

In [15]:
# from typing import List

# from sflkit.analysis.analysis_type import AnalysisType
# from sflkit.analysis.spectra import Line

# analyzer2 = analyzer.analyze()
# print("hi")
# coverage: List[Line] = analyzer2.get_coverage(AnalysisType.LINE)
# coverage = {line.line for line in coverage}
# print(coverage)
from avicenna.avix import *
from avicenna.oracle_construction import * 
avix_oracle = construct_oracle(
                            program_under_test='main',
                            # inp_converter=None,
                            # timeout=10,
                            line = 18,
                            put_path='calculator.py'
                            # resource_path='rsc/',
                            # program_oracle= None,
                            )


In [16]:
avix_oracle('sqrt(9)')


<OracleResult.FAILING: 'FAILING'>

In [17]:
avix_oracle('sin(9)')

<OracleResult.PASSING: 'PASSING'>

In [18]:
def calc_oracle(inp):
    # convert inputs
    
    return eval(inp)
    
    inputs = input.split()
    result_list = [x, y, z]
    result_list.sort()
    return result_list[1]

In [19]:
import math
calc_oracle('math.sqrt(9)')

3.0

In [20]:
regular_oracle = construct_oracle(
                            program_under_test=main,
                            timeout=10,
                            program_oracle=calc_oracle,
                            default_oracle_result=OracleResult.PASSING)



In [21]:
inputs_calc = ['sqrt(9)','cos(8)']

In [22]:
avix_oracle(inputs_calc[0])

<OracleResult.FAILING: 'FAILING'>

In [23]:
# avicenna = Avicenna(
#     grammar=grammar_calc,
#     initial_inputs=inputs_calc,
#     oracle=avix_oracle,
#     max_iterations=10,
#     top_n_relevant_features=3,
# )

In [24]:
avix = AviX(
    grammar=grammar_calc,
    initial_inputs=inputs_calc,
    oracle=avix_oracle,
    max_iterations=10,
    top_n_relevant_features=3,
    put_path='calculator.py'
)

sflkit :: INFO     :: I found 16 events in calculator.py.
sflkit :: INFO     :: I found 16 events in calculator.py.


In [25]:
# avicenna2 = Avicenna(
#     grammar=middle_grammar,
#     initial_inputs=middle_inputs,
#     oracle=regular_oracle,
#     max_iterations=10,
#     top_n_relevant_features=3, 
# )

In [26]:
import warnings

import logging
# Suppress the specific SHAP warning
warnings.filterwarnings(
    "ignore",
    message="LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray",
)
warnings.filterwarnings(
    "ignore", 
    message="No further splits with positive gain, best gain: -inf"
)

In [27]:
logging.basicConfig(filename='calc.log', filemode='w', encoding='utf-8', level=logging.INFO, force=True)
# only 2 constraints used in the end why
best_invariant = avix.explain() # unparse with islaunparse for further use


KeyboardInterrupt: 

In [ ]:
#another_invariant = avicenna2.explain() # unparse with islaunparse for further use

In [ ]:
#print(another_invariant)

In [ ]:
print(best_invariant)
print(best_invariant[0])
print(best_invariant[2])

(ConjunctiveFormula(ForallFormula(BoundVariable("elem_1", "<number>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<one_nine>"), Constant("start", "<start>"), SMTFormula('(>= (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<number>"), BoundVariable("elem_2", "<one_nine>"), ))), ExistsFormula(BoundVariable("elem", "<function>"), Constant("start", "<start>"), SMTFormula('(= elem "sqrt")', BoundVariable("elem", "<function>"), ))), 1.0, 1.0)
(∀ elem_1 ∈ start: (∃ elem_2 ∈ start: (StrToInt(elem_1) >= StrToInt(elem_2))) ∧ ∃ elem ∈ start: (elem == "sqrt"))
1.0


In [ ]:
# solver1 = ISLaSolver(
#     grammar=grammar_calc,
#     formula=another_invariant[0],
#     enable_optimized_z3_queries=False,
# )
from isla.solver import ISLaSolver

solver2 = ISLaSolver(
    grammar=grammar_calc,
    formula=best_invariant[0],
    enable_optimized_z3_queries=False,
)


In [ ]:
grammar_calc = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><one_nine><maybe_digits><maybe_frac>", 
                 "-0",
                 "0"],
    "<maybe_minus>": ["",
                      "-"
                      ],
    "<maybe_frac>": ["", ".<digits>"],
    "<one_nine>": [str(num) for num in range(1, 10)],
    "<digit>": [digit for digit in string.digits],
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
}

In [ ]:
solver2.formula

ConjunctiveFormula(ForallFormula(BoundVariable("elem_1", "<number>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<one_nine>"), Constant("start", "<start>"), SMTFormula('(>= (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<number>"), BoundVariable("elem_2", "<one_nine>"), ))), ExistsFormula(BoundVariable("elem", "<function>"), Constant("start", "<start>"), SMTFormula('(= elem "sqrt")', BoundVariable("elem", "<function>"), )))

In [ ]:
results2 = []

# # print(best_invariant[0])
# # print(another_invariant[0])
# for _ in range(1,10):
#     results1.append(solver1.solve())
    
#    avix_oracle()
for _ in range(1,10):
    results2.append(solver2.solve())

StopIteration: 

In [ ]:
for input in results2:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))

In [ ]:
for input in results2:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))
for input in results2:
    print(input)

In [ ]:
solver = ISLaSolver(
    grammar=middle_grammar,
    formula=best_invariant[0],
    enable_optimized_z3_queries=False,
)

# this isnt working rn why not raaaaaaaaaaaaaaaaaaaaaaa
# should be inputs of type 2, 3, 1
for _ in range(1,10):
    print(solver.solve())


NameError: name 'middle_grammar' is not defined

In [ ]:
# call func for middle, converts string input to usable integer values

def call_func_middle(inp: str):
    
    inp = inp.__str__()
    
    middle_input = inp.split(',')
    converted_inp =  [int(middle_input[0]), int(middle_input[1]), int(middle_input[2])]
    return converted_inp

In [ ]:
test_path = Path('rsc/')
print(Path.exists(test_path))


True


In [ ]:
from avicenna.avix import *
from avicenna.oracle_construction import * 

In [ ]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [ ]:
import tmp
importlib.reload(tmp)
tmp.sflkitlib.lib.reset()
avix_oracle = construct_oracle(
                            program_under_test='middle',
                            inp_converter=middle_inp_conv,
                            timeout=10,
                            line = 7,
                            resource_path='rsc/',
                            program_oracle= None)


In [ ]:
avix_oracle("6,3,7")

<OracleResult.FAILING: 'FAILING'>

In [ ]:
# # double check if this works

# AviX.create_event_file(instrumented_function='middle',
#                        #instr_path='tmp',
#                        inp = '2,1,3',
#                        conversion_func=middle_inp_conv,
#                        event_path='rsc/event_file'
#                        )

In [ ]:
avix = AviX(grammar=middle_grammar,
            initial_inputs=middle_inputs,
            oracle=avix_oracle,
            max_iterations=10,
            desired_line=7,
            put_path='middle.py',
            #instr_path='instrumented.py',
            min_precision = 0.7,)

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer, DerivationTree

middle_grammar_converted = {
    "<start>": ["<stmt>"],
    "<stmt>": ["str.to.int(<x>),str.to.int(<y>),str.to.int(<z>)"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

fuzzer = GrammarFuzzer(middle_grammar)

for i in range(10):
    print(fuzzer.fuzz())

14,977,74
9,1,1
86,87619,98
75,88,2
4,3,7
4,8,74
67,11,97
6,25,182
7958,6,82
5,46,2


https://github.com/uds-se/sflkit

<img src="qrcode.png" style="width:500px">